In [250]:
import torch
import pickle
import random

from sklearn.decomposition import PCA



class close_words:
    def __init__(self,model,w2i,pca_1=False,n=5):
        self.model=model
        self.w2i=w2i
        self.n=n
        self.i2w=dict(zip(w2i.values(),w2i.keys()))
        pca = PCA(n_components=10)
        self.embeddings=model['embedding_in.weight'].detach() if not pca_1 else torch.tensor(pca.fit_transform(model['embedding_in.weight'].detach()))
 
    def input_word(self,word):
        self.sample_idx=self.w2i[word]
        self.sample_vec=self.embeddings[self.sample_idx]
    
    def sample_word(self):
        
        self.sample_idx=random.sample(list(self.w2i.values()),1)[0]
        self.sampled_word=self.i2w[self.sample_idx]
        self.sample_vec=self.embeddings[self.sample_idx]
        print('임의의 단어:',self.sampled_word)
    
    def l2_dist(self):
        
        trial=torch.mul((self.embeddings-self.sample_vec),(self.embeddings-self.sample_vec))
        trial[self.sample_idx]=100    
        trial=torch.sum(trial,dim=1)
        #rec=torch.argmax(-self.trial).item()
        idexes=torch.argsort(-trial, descending=True)[:self.n]
        step=1
        print('사전사이즈:',len(self.i2w))
        for idx in idexes:
            if idx==len(w2i): #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
                
    def cos_sim(self):
        
   
        upper=torch.sum(torch.mul(self.embeddings,self.sample_vec),dim=1)
  
        lower=torch.sum(torch.mul(self.embeddings,self.embeddings),dim=1)+torch.sum(torch.mul(self.sample_vec,self.sample_vec))
    
        trial=torch.div(upper,lower)

        trial[self.sample_idx]=-100
        idexes=torch.argsort(trial, descending=True)[:self.n]
        step=1
        for idx in idexes:
            if idx==len(w2i)+1: #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
        

In [251]:
#model=torch.load('./w2v_withoud_ns',map_location='cpu')
model=torch.load('./training_with_mecab_2018_2_epoch_14',map_location='cpu')['state_dict']

with open('./preprocessing//vocab_mecab.pickle','rb') as f:
    w2i=pickle.load(f)
    
print('임베딩 사이즈:',model['embedding_in.weight'].shape)  #unk 랑 embedding이 0부터인것을 몰랐음. 0은 버리면 됨
print('사전 사이즈:',len(w2i))  

임베딩 사이즈: torch.Size([30655, 256])
사전 사이즈: 30654


In [256]:
showing=close_words(model,w2i,pca_1=True,n=5)
showing.sample_word()
showing.l2_dist()
print('임의의 단어',showing.sampled_word)
showing.cos_sim()

임의의 단어: ('제목', 'NNG')
사전사이즈: 30654
1 번째 가까운 단어: ('자본', 'NNG')
2 번째 가까운 단어: ('문사', 'NNG')
3 번째 가까운 단어: ('맛보기', 'NNG')
4 번째 가까운 단어: ('철두철미', 'MAG')
5 번째 가까운 단어: ('니까요', 'VCP+EC')
임의의 단어 ('제목', 'NNG')
1 번째 가까운 단어: ('맛보기', 'NNG')
2 번째 가까운 단어: ('문사', 'NNG')
3 번째 가까운 단어: ('자본', 'NNG')
4 번째 가까운 단어: ('영양소', 'NNG')
5 번째 가까운 단어: ('익힌', 'VV+ETM')


In [268]:
showing.input_word(('과제', 'NNG'))
showing.l2_dist()
print('====')
showing.cos_sim()

사전사이즈: 30654
1 번째 가까운 단어: ('있', 'VA')
2 번째 가까운 단어: ('없', 'VA')
3 번째 가까운 단어: ('강의', 'NNG')
4 번째 가까운 단어: ('문제', 'NNG')
5 번째 가까운 단어: ('같', 'VA')
====
1 번째 가까운 단어: ('있', 'VA')
2 번째 가까운 단어: ('없', 'VA')
3 번째 가까운 단어: ('강의', 'NNG')
4 번째 가까운 단어: ('않', 'VX')
5 번째 가까운 단어: ('같', 'VA')


## UNK 빈도수

In [98]:
with open('./preprocessing/vocab_without_josa_gut_su_freq.pickle','rb') as f:
    total_freq=pickle.load(f)
    
with open('./preprocessing/vocab_without_josa_gut_su.pickle','rb') as h:
    voc=pickle.load(h)
    
not_unk=list(voc.keys())

freq_sum=0

def summing(x):
    
    global freq_sum
    freq_sum+=x

_=list(map(lambda x:summing(total_freq[x]),not_unk))

ratio=(sum(total_freq.values())-freq_sum)/sum(total_freq.values())
print(ratio)

0.014246669941204219


In [102]:
## unk 빈도수가 문제는 아닌듯?? 

In [7]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: FEDA-124D

 C:\tensor_code\kluebot\Recommendation_System\embedding_re 디렉터리

2019-12-24  오후 12:20    <DIR>          .
2019-12-24  오후 12:20    <DIR>          ..
2019-12-23  오후 07:18    <DIR>          .ipynb_checkpoints
2019-12-22  오후 10:16             1,601 doc2vec_model.py
2019-12-22  오후 03:13        54,120,715 doc2vec_training_set_2017_1.pickle
2019-12-23  오후 07:18    <DIR>          preprocessing
2019-12-24  오후 12:20             7,241 sampling_sim.ipynb
2019-12-21  오후 05:07    <DIR>          skipgram_dataset
2019-12-23  오후 07:18             4,834 train_doc.py
2019-12-22  오후 03:22             5,821 train_ns.py
2019-12-18  오전 11:43             2,399 w2v_ns.py
2019-12-23  오전 12:28    <DIR>          __pycache__
               6개 파일          54,142,611 바이트
               6개 디렉터리   9,386,196,992 바이트 남음


In [8]:
import pandas as pd
pd.read_csv('../recommendation/tfidf/lecture_vector_2017_1.csv').tail()

,0,1,2,3,4,5,6,7,8,9,...,189,190,191,192,193,194,195,196,197,198
2684,"('박성근 교수님', '물리교과교육론')","('그냥', 'MAG') ('아무것', 'NNG') ('안', 'MAG') ('했'...",0.213964,0.0,0.000000,0.0,0.000000,0.220355,0.133806,0.000000,...,0.097532,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2685,"('김지혜 교수님', '화학교과교육론')","('초반', 'NNG') ('교수', 'NNG') ('수업', 'NNG') ('진행...",0.000000,0.0,0.000000,0.0,0.000000,0.097078,0.000000,0.000000,...,0.171871,0.0,0.072503,0.0,0.0,0.0,0.0,0.0,0.095118,0.0
2686,"('한신 교수님', '지구과학교과교육론')","('학습', 'NNG') ('량', 'NNG') ('보통', 'NNG') ('이'...",0.000000,0.0,0.000000,0.0,0.157612,0.000000,0.000000,0.000000,...,0.106368,0.0,0.179482,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2687,"('성국현 교수님', '군사학Ⅰ')","('고려', 'NNG') ('대학교', 'NNG') ('학우', 'NNG') ('라...",0.000000,0.0,0.712342,0.0,0.000000,0.000000,0.000000,0.276746,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2688,"('이보선 교수님', '군사학Ⅲ')","('참으로', 'MAG') ('의미', 'NNG') ('깊', 'VA') ('수업'...",0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
